# Shaple value tutorial

In the methods for the explanability and interpretibility of machine learning models, shapley-value based methods are one commonly used class with theoretical guarantee. In this tutorial, we will mainly focus on the package SHAP and  
1. reimplement the simplified Kernel SHAP for understanding how it works and how to use the package for applications;
2. apply Kernel SHAP to explaining the results of a neural network in a regression task;
3. apply Kernel SHAP to the results of sklearn models on the iris dataset;
4. apply Kernel SHAP to explaining the sentiment analysis results.

The tutorial is mainly adapted from the official documents of [flax](https://flax.readthedocs.io/en/latest/notebooks/flax_basics.html) and [shap](https://github.com/slundberg/shap#citations). 

After finishing 3 and 4, one can be able to apply shap to applications. A complete of 3 and 4 can be awarded with the basic points.

After finishing 1 and 2, one can be able to understand the basics of Kernel SHAP and build neural networks with flax. A complete of 1 and 2 can be awarded with bonus.

## Install shap package
The [shap package](https://github.com/slundberg/shap) is in general based on the work   
* [Lundberg, Scott M., and Su-In Lee. "A unified approach to interpreting model predictions." Proceedings of the 31st international conference on neural information processing systems. 2017.](https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf)

In [ ]:
!pip install shap

## PART I: Kernel SHAP
In this tutorial, we focus on Kernel SHAP which approximates shapely values. The computation mehtods has been introduced in Theorem 2 of [(Lundberg, Scott M., and Su-In Lee, 2017)](https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf).
Interestingly, one can implement the method with weighted linear regression. In this part,
* We first apply Kernel SHAP in shap to a simple linear function;
* We then reimplement Kernel SHAP from scratch and compare the result with Kernel SHAP in shap;
* Finally, we build a neural network for a simple (non)-linear regression task and apply Kernel SHAP to explain its results.


### Kernel SHAP for linear functions

* Input $\mathbf{x}_i$ is a four-dimensional vector;
* The unknown function is $f(\mathbf{x}_i) = \mathbf{\beta} \cdot \mathbf{x}_i + 10$, where $\beta$ and $10$ are the function parameters.

In [ ]:
import numpy as np
import shap

In [ ]:
M = 4
np.random.seed(1)
x = np.random.randn(M)
def f(X):
    np.random.seed(0)
    beta = np.random.rand(X.shape[-1])
    return np.dot(X,beta) + 10
f(x)

### Create a explainer with Kernel SHAP in shap
Please fill in this code block to create a explainer with kernel SHAP in [the shap package](https://github.com/slundberg/shap#citationshttps://github.com/slundberg/shap#citations).

In [ ]:
# explainer = ...

Please fill in this code block to print the shap values for explaining the $f(\mathbf{x}_i)$ where the input $\mathbf{x}_i$ and the function parameters $\beta$ are randomly generated.

In [ ]:
# shap_values = ...

print("shap_values =", shap_values)
print("base value =", explainer.expected_value)

### Implement Kernel SHAP with weighted linear regression

Please reimplement Kernel SHAP from scratch. You may need to consider Theorem 2 of [(Lundberg, Scott M., and Su-In Lee, 2017)](https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf).

In [ ]:
#### Please finish the function ####
def kernel_shap(...):
    ...
    return base_value, shap_values
#### END ####

M = 4
np.random.seed(1)
x = np.random.randn(M)

#### Call the function ####
phi = kernel_shap(...)
#### END ####

base_value = phi[-1]
shap_values = phi[:-1]

print("  reference =", reference)
print("          x =", x)
print("shap_values =", shap_values)
print(" base_value =", base_value)
print("   sum(phi) =", np.sum(phi))
print("       f(x) =", f(x))

## Kernel SHAP for black box methods
In this part, we will 
* first generate synthetic datasets X ( a set of four-dimensional vectors) and Y (regressopm target);
* then, build a simple neural network with [jax](https://github.com/google/jax) and [flax](https://github.com/google/flax).
* finally, apply KPernel SHAP to the results of the neural network.

In [ ]:
# Install jax
!pip install jax

In [ ]:
# Install flax
!pip install --upgrade pip jax jaxlib
!pip install --upgrade git+https://github.com/google/flax.git

In [ ]:
import flax.linen as nn
import jax
from jax import value_and_grad,lax, random, numpy as jnp
from typing import Any, Callable, Sequence, Optional
from flax.core import freeze, unfreeze
from flax import optim
import optax

### Step 1: Generate synthetic data

$ y = W \cdot \sin(\mathbf{x}) + b + 0.1 \epsilon,$
where $W$ and $b$ are function parameters and it is a nonlinear function because of $\sin(\cdot)$.

In [ ]:
# Set problem dimensions
nsamples = 5000
xdim = 4
ydim = 1

# Generate random ground truth W and b
key = random.PRNGKey(0)
k1, k2 = random.split(key)
W = random.normal(k1, (xdim, ydim))
b = 10.0
true_params = freeze({'params': {'bias': b, 'kernel': W}})

# Generate samples with additional noise
ksample, knoise = random.split(k1)
x_samples = random.normal(ksample, (nsamples, xdim))

y_samples = jnp.dot(jnp.sin(x_samples), W) + b
y_samples += 0.1*random.normal(knoise,(nsamples, ydim)) # Adding noise
print('x shape:', x_samples.shape, '; y shape:', y_samples.shape)

### Step 2: Create a neural network class

In [ ]:
class BBNN(nn.Module):
    features: Sequence[int]

    def setup(self):
    # we automatically know what to do with lists, dicts of submodules
        self.layers = [nn.Dense(feat) for feat in self.features]
    def __call__(self, inputs):
        x = inputs
        
        for i, lyr in enumerate(self.layers):
            x = lyr(x)
            if i != len(self.layers) - 1:
                x = nn.relu(x)
        return x

### Step 3: Define a loss function

Please fill in this code block with your loss function (you may consider least square error for training on this simple scenario).

In [ ]:
# def loss(...):
#     return ...

### Step 4: Initialize a neural network and train it with the dataset
One may consider to split the dataset into training and testing datasets and then train with batches.  
One could also consider to train with all data as linear regression, since this is a relatively simple case.

In [ ]:
num_feature = xdim

key = random.PRNGKey(0)
key, subkey = random.split(key)

x = np.random.randn(num_feature)
model = BBNN(features=[2, 5,1])  # define a customized neural network
params = model.init(subkey, x)
y = model.apply(params, x)

One could implememnt gradient descent for training as in linear regression.  
One could also use the built-in optimizer in flax, such as [optax](#https://flax.readthedocs.io/en/latest/notebooks/flax_basics.html#Optimizing-with-Optax).  
Please fill in the training process in the following code block: 

In [ ]:
##### Training #####

##### END #####

### Step 5:  define a prediction function

Prediction function

In [ ]:
def f_NN(X,params=params,model=model):
    return model.apply(params, X)

Ground-true function

In [ ]:
def f(X,W=W,b=b):
    W = np.array(W.reshape(-1))
    return np.dot(np.sin(X), W) + b

Explain the prediction function f_NN with Kernal SHAP in shap:  
Please fill in the following code block:

In [ ]:
x = np.array(x_samples[0,:])

######## create an explainer with kernel SHAP ########
explainer = ...
shap_values = ...
######## END ########

print("input =", x)
print("output =",f_NN(x))
print("ground truth =",f(x))
print("shap_values =", shap_values)
print("base value =", explainer.expected_value)

### Step 6: compare with Kernel SHAP

In [ ]:
######## create an explainer with kernel SHAP ########


explainer = ...
shap_values = ...
######## END ########
print("input =", x)
print("output =",f(x))
print("shap_values =", shap_values)
print("base value =", explainer.expected_value)

## PART II: Application of Kernel SHAP


### Tabular dataset

Load iris dataset with sklearn

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
import time

X_train,X_test,Y_train,Y_test = train_test_split(*shap.datasets.iris(), test_size=0.2, random_state=0)

# rather than use the whole training set to estimate expected values, we could summarize with
# a set of weighted kmeans, each weighted by the number of points they represent. But this dataset
# is so small we don't worry about it
#X_train_summary = shap.kmeans(X_train, 50)

def print_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

shap.initjs()

Please use the model "KNeighborsClassifier" in sklearn and fit the model with the training dataset,

In [ ]:
###### define a model ######
model = ...
###### END ######

model.fit(X_train, Y_train)

Please create a explainer for the model with Kernel SHAP:

In [ ]:
explainer = ...
shap_values = ...

Please draw the force plot of the shap values for the first sample "X_test.iloc[0,:]"

In [ ]:
shap.force_plot(...)

Please draw all the test data with force plot

In [ ]:
shap_values = ...
shap.force_plot(...)

Please try "summery_plot" in shap

In [ ]:
shap.summary_plot(...)

### Use shap.explainer

One can not only use kernel shap but also other explainers. In this part we try shap.explainer:

In [ ]:
f = lambda x: model.predict_proba(x)[:,1]
med = X_train.median().values.reshape((1,X_train.shape[1]))

explainer = shap.Explainer(f, med)
shap_values = explainer(X_test)

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.scatter(shap_values[:,"petal length (cm)"])

### Sentiment analysis
Classify the "positive" and "negative" sentiment of IMDB reviews.  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
shap.initjs()

**Load the dataset of IMDB reviews** 

In [ ]:
corpus,y = shap.datasets.imdb()
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=7)
corpus_train, corpus_test, y_train, y_test = corpus_train[:10000],corpus_test[:500],y_train[:10000],y_test[:500]

**Preprocess and Create training and testing dataset**

In [ ]:
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(corpus_train).toarray() 
X_test = vectorizer.transform(corpus_test).toarray()

**Define a logistic classification model.** 

In [ ]:
model = sklearn.linear_model.LogisticRegression(penalty="l2", C=0.1)
model.fit(X_train, y_train)

**Build explainer**  
Please fill in your answer in the following code block.

In [ ]:
###### Please fill in your answer here ######
explainer = shap.Explainer(...)
shap_values = ...
###### END ######

Please plot 10 features with beeswarm 

In [ ]:
shap.plots.beeswarm(...)

In [ ]:
ind = 10
print("Positive" if y_test[ind] else "Negative", "Review:")
print(corpus_test[ind])

Please plot the force plot for the review, corpus_test[10].

In [ ]:
shap.plots.force(...)

Please also try the bar plot

In [ ]:
shap.plots.bar(...)

# END